In [ ]:
# Celda 1: Configuración e Importaciones

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Añadir el directorio raíz del proyecto al sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Usaremos nuestra clase de conexión existente para ser consistentes
from src.database import DatabaseConnection

print("Entorno configurado. Listo para el experimento de limpieza de datos.")

In [ ]:
# Celda 2: Extracción de Datos Crudos (CORREGIDA)

print("Conectando a la base de datos y extrayendo la tabla 'sales'...")

engine = DatabaseConnection().get_engine()

try:
    with engine.connect() as connection:
        # Corregimos el nombre de la columna en la consulta SQL
        sales_raw_df = pd.read_sql("SELECT SalesID, salesdate FROM sales", connection)
    
    # ¡IMPORTANTE! Renombramos la columna en Pandas para que el resto del notebook funcione
    # sin necesidad de más cambios.
    sales_raw_df.rename(columns={'salesdate': 'SalesDate'}, inplace=True)

    print(f"Se extrajeron {len(sales_raw_df)} registros de la tabla 'sales'.")
    display(sales_raw_df.head())
except Exception as e:
    print(f"Error al extraer los datos: {e}")

In [ ]:
# Celda 3: Hipótesis de Reconstrucción de Fechas (Formato HH:MM.f)

print("--- INICIO DE LA CELDA 3 ---")

if 'sales_raw_df' not in locals() or sales_raw_df.empty:
    print("ERROR: El DataFrame 'sales_raw_df' no existe o está vacío. Ejecuta la Celda 2 primero.")
else:
    print(f"DataFrame 'sales_raw_df' encontrado con {len(sales_raw_df)} filas.")
    
    # --- HIPÓTESIS ---
    # La fecha es un offset en formato 'HH:MM.f' (Horas, Minutos y fracción de minuto)
    base_datetime_str = '2023-01-01 00:00:00' # Empezamos a medianoche para que las horas sumen limpiamente
    base_datetime = pd.to_datetime(base_datetime_str)
    print(f"Fecha base establecida: {base_datetime_str}")

    # --- FUNCIÓN DE TRANSFORMACIÓN MANUAL ---
    def convert_hh_mm_to_timedelta(time_str):
        """
        Convierte un string en formato 'HH:MM.f' a un objeto Timedelta.
        'f' es una fracción de minuto (ej. .2 = 12 segundos).
        """
        if not isinstance(time_str, str):
            return pd.NaT

        try:
            # 1. Dividir horas de minutos
            parts = time_str.split(':')
            if len(parts) != 2:
                return pd.NaT # Formato incorrecto si no hay un ':'
            
            hours_str = parts[0]
            minutes_and_fraction_str = parts[1]

            # 2. Dividir minutos de la fracción
            if '.' in minutes_and_fraction_str:
                min_parts = minutes_and_fraction_str.split('.')
                minutes_str = min_parts[0]
                # La fracción es sobre 10, 100, etc. La convertimos a un float
                fraction_str = min_parts[1]
                fraction_of_minute = float(f"0.{fraction_str}")
            else:
                minutes_str = minutes_and_fraction_str
                fraction_of_minute = 0.0

            # 3. Convertir a números
            hours = int(hours_str)
            minutes = int(minutes_str)
            
            # 4. Calcular los segundos totales de la fracción
            seconds_from_fraction = fraction_of_minute * 60
            
            # 5. Crear el Timedelta final
            return pd.to_timedelta(hours, unit='h') + \
                   pd.to_timedelta(minutes, unit='m') + \
                   pd.to_timedelta(seconds_from_fraction, unit='s')

        except (ValueError, TypeError, IndexError):
            # Si cualquier conversión o división falla, es un formato inválido
            return pd.NaT

    # --- APLICACIÓN Y TRANSFORMACIÓN ---
    print("\nPaso 1: Aplicando la función de transformación 'HH:MM.f' a 'SalesDate'...")
    sales_raw_df['TimeOffset'] = sales_raw_df['SalesDate'].apply(convert_hh_mm_to_timedelta)
    
    # --- VALIDACIÓN INTERMEDIA ---
    error_count = sales_raw_df['TimeOffset'].isna().sum()
    print(f"-> Se encontraron {error_count} valores que no pudieron ser convertidos.")

    # 2. Creamos la nueva columna de fecha reconstruida.
    print("\nPaso 2: Sumando la fecha base y el offset de tiempo...")
    sales_raw_df['SalesDate_reconstructed'] = base_datetime + sales_raw_df['TimeOffset']
    
    print("\n--- TRANSFORMACIÓN COMPLETADA ---")

    # --- ANÁLISIS DE RESULTADOS ---
    total_count = len(sales_raw_df)
    print(f"\n - Registros totales: {total_count}")
    print(f" - Registros procesados exitosamente: {total_count - error_count}")

    # Mostramos algunos de los registros que SÍ se procesaron
    print("\nEjemplos de registros procesados correctamente:")
    display(sales_raw_df[sales_raw_df['TimeOffset'].notna()].head())

print("\n--- FIN DE LA CELDA 3 ---")

In [ ]:
# Celda 4: Comparación de la Reconstrucción

print("Mostrando una comparación de la columna original y la fecha reconstruida...")

# Creamos un DataFrame de comparación
reconstruction_df = sales_raw_df[['SalesDate', 'TimeOffset', 'SalesDate_reconstructed']].copy()
reconstruction_df['is_error'] = sales_raw_df['TimeOffset'].isna()

# Mostramos una muestra de casos que SÍ se pudieron convertir
print("\n--- Ejemplos de reconstrucción exitosa ---")
display(reconstruction_df[reconstruction_df['is_error'] == False].head())

# Mostramos los casos que NO se pudieron convertir
if error_count > 0:
    print("\n--- Ejemplos de registros que fallaron en la reconstrucción ---")
    display(reconstruction_df[reconstruction_df['is_error'] == True].head())